In [1]:
import json
from websocket import WebSocketApp
from graph_utils import  insert_transaction
from config import BLOCKCHAIN_WS_URL,BLOCKSTREAM_API

DATA LOADING

Note - Loaded only one Block of APR20 data. The Loaded data is stored in load_backup_to_db file.

In [ ]:
import requests
import time
import json


# Convert your date range to UNIX timestamps
START_DATE = "2025-04-20"
END_DATE = "2025-04-20"

# Helper: convert date to unix timestamp
def date_to_timestamp(date_str):
    return int(time.mktime(time.strptime(date_str, "%Y-%m-%d")))

start_timestamp = date_to_timestamp(START_DATE)
end_timestamp = date_to_timestamp(END_DATE) + 86399  # include the whole end day

def get_block_height_by_timestamp(target_ts, search_direction='up'):
    """
    Find the first block >= (or <= if search_direction='down') target_ts using binary search.
    """
    # Get latest block height
    tip_height = int(requests.get(f"{BLOCKSTREAM_API}/blocks/tip/height").text)
    low, high = 0, tip_height

    while low <= high:
        mid = (low + high) // 2
        r = requests.get(f"{BLOCKSTREAM_API}/block-height/{mid}")
        if r.status_code != 200:
            break
        block_hash = r.text
        block = requests.get(f"{BLOCKSTREAM_API}/block/{block_hash}").json()
        block_time = block['timestamp']
        if block_time < target_ts:
            low = mid + 1
        elif block_time > target_ts:
            high = mid - 1
        else:
            return mid
    return low if search_direction == 'up' else high

def get_block_hash(height):
    r = requests.get(f"{BLOCKSTREAM_API}/block-height/{height}")
    if r.status_code == 200:
        return r.text
    return None

def get_block_time(block_hash):
    r = requests.get(f"{BLOCKSTREAM_API}/block/{block_hash}")
    if r.status_code == 200:
        return r.json()['timestamp']
    return None

def get_block_txids(block_hash):
    r = requests.get(f"{BLOCKSTREAM_API}/block/{block_hash}/txids")
    if r.status_code == 200:
        return r.json()
    return []

def get_tx(txid):
    r = requests.get(f"{BLOCKSTREAM_API}/tx/{txid}")
    if r.status_code == 200:
        return r.json()
    return None

def filter_and_format_tx(tx):
    # Build the structure as specified
    return {
        "txid": tx["txid"],
        "status": tx["status"],
        "vin": [
            {
                "prevout": {
                    "scriptpubkey_address": vin["prevout"].get("scriptpubkey_address"),
                    "value": vin["prevout"].get("value")
                } if vin.get("prevout") else {}
            } for vin in tx.get("vin", [])
        ],
        "vout": [
            {
                "scriptpubkey_address": vout.get("scriptpubkey_address"),
                "value": vout.get("value")
            } for vout in tx.get("vout", [])
        ]
    }

def main():
    print("Locating block heights for your date range...")
    start_height = get_block_height_by_timestamp(start_timestamp, 'up')
    end_height = get_block_height_by_timestamp(end_timestamp, 'down')
    print(f"Block heights from {start_height} to {end_height}")

    all_txs = []
    for height in range(893204, 893204 + 1): # Processing only for one block by hardcoding the value, later it'll be done for multiple blocks
        block_hash = get_block_hash(height)
        if not block_hash:
            continue
        block_time = get_block_time(block_hash)
        if not (start_timestamp <= block_time <= end_timestamp):
            continue
        print(f"Processing block {height} at {block_time}...")
        txids = get_block_txids(block_hash)
        cnt=0
        for txid in txids:
            tx = get_tx(txid)
            if tx:
                tx_struct = filter_and_format_tx(tx)
                all_txs.append(tx_struct)
            cnt+=1
            print(f"{cnt}/{len(txids)}")
        print(tx_struct)
        time.sleep(0.2)  # be nice to the API

    # Write to JSON file
    with open("bitcoin_transactions_backup.json", "w", encoding='utf-8') as f:
        json.dump(all_txs, f, indent=2, ensure_ascii=False)

    print(f"Saved {len(all_txs)} transactions to bitcoin_transactions_backup.json")

if __name__ == "__main__":
    main()


Loaded 7163 transactions from backup file.
Found 518 unconfirmed transactions.


 98%|█████████▊| 507/518 [05:40<00:07,  1.54it/s]

Request failed: 404 Client Error: Not Found for url: https://blockstream.info/api/tx/590e44ee0fada63174b35ab87c19ea41d2d0a5184126ad6161aeb65f42feb0aa. Retrying in 1 seconds...
Request failed: 404 Client Error: Not Found for url: https://blockstream.info/api/tx/590e44ee0fada63174b35ab87c19ea41d2d0a5184126ad6161aeb65f42feb0aa. Retrying in 2 seconds...
Request failed: 404 Client Error: Not Found for url: https://blockstream.info/api/tx/590e44ee0fada63174b35ab87c19ea41d2d0a5184126ad6161aeb65f42feb0aa. Retrying in 4 seconds...
Request failed: 404 Client Error: Not Found for url: https://blockstream.info/api/tx/590e44ee0fada63174b35ab87c19ea41d2d0a5184126ad6161aeb65f42feb0aa. Retrying in 8 seconds...
Request failed: 404 Client Error: Not Found for url: https://blockstream.info/api/tx/590e44ee0fada63174b35ab87c19ea41d2d0a5184126ad6161aeb65f42feb0aa. Retrying in 16 seconds...


100%|█████████▉| 517/518 [06:18<00:01,  1.08s/it]

Request failed: 404 Client Error: Not Found for url: https://blockstream.info/api/tx/36215607fa1d6aaf88ab70cff6e5e3d0b59ccd263ffa782609c06abe279f6324. Retrying in 1 seconds...
Request failed: 404 Client Error: Not Found for url: https://blockstream.info/api/tx/36215607fa1d6aaf88ab70cff6e5e3d0b59ccd263ffa782609c06abe279f6324. Retrying in 2 seconds...
Request failed: 404 Client Error: Not Found for url: https://blockstream.info/api/tx/36215607fa1d6aaf88ab70cff6e5e3d0b59ccd263ffa782609c06abe279f6324. Retrying in 4 seconds...
Request failed: 404 Client Error: Not Found for url: https://blockstream.info/api/tx/36215607fa1d6aaf88ab70cff6e5e3d0b59ccd263ffa782609c06abe279f6324. Retrying in 8 seconds...
Request failed: 404 Client Error: Not Found for url: https://blockstream.info/api/tx/36215607fa1d6aaf88ab70cff6e5e3d0b59ccd263ffa782609c06abe279f6324. Retrying in 16 seconds...


100%|██████████| 518/518 [06:50<00:00,  1.26it/s]

Updated {updated_count} out of {unconfirmed_txs_count} unconfirmed transactions
